In [4]:
import numpy as np
#import math

import torch

from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [22]:
# -------preprocessing of data ---------
# x1 = list(range(len(traindata)))
# np.random.shuffle(x1)
# x2 = list(range(len(testdata)))
# np.random.shuffle(x2)

# traindata1 = traindata[np.argsort(x1)]
# trainlabels1 = trainlabels[np.argsort(x1)]
# testdata1 = testdata[np.argsort(x2)]
# testlabels1 = testabels[np.argsort(x2)]

# gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
# #np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
# # from PIL import Image

# # # for td in traindata:
# # td = traindata[10]
# # print(td.T.shape)
# # im = Image.fromarray(td.T)
# # im.save('out.jpg')
# # #im.convert('RGB').save('out.jpg')
# #   #im.show()
  
# # td = np.dot(td.T,[0.2125, 0.7154, 0.0721])
# #   # print(td[0])
# # img = Image.fromarray(td)

# # img.convert('RGB').save('out.png')

# greydata = []
# #i = 0
# for td in testdata1:
#     #print(td.shape)
#     greydata.append(np.dot(td.T,[0.2125, 0.7154, 0.0721]).flatten())

# greydata = np.asarray(greydata)

# np.save(datapath+"grey_testdata.npy",greydata)
# np.save(datapath+"grey_testlabel.npy",testlabels1)

In [6]:
# # BATCH_SIZE = 4

# # transform = transforms.ToTensor()

# # traindata = torchvision.datasets.MNIST('/tmp', train=True, download=True, transform=transform)
# # trainloader = torch.utils.data.DataLoader(traindata, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# # testdata = torchvision.datasets.MNIST('/tmp', train=False, download=True, transform=transform)
# # testloader = torch.utils.data.DataLoader(testdata, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
# dataiter = iter(testloader)

# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0,0,0), (1,1,1))]) #change normalize values if need be

# # svhn = torchvision.datasets.SVHN('/tmp', download=True, transform=transform)
# mnist = torchvision.datasets.MNIST('/tmp', download=True, transform=transform)

# # svhn_loader = torch.utils.data.DataLoader(dataset=svhn,
# #                                           batch_size=1,
# #                                           shuffle=True,
# #                                           num_workers=2)

# mnist_loader = torch.utils.data.DataLoader(dataset=mnist,
#                                             batch_size=4,
#                                             shuffle=True,
#                                             num_workers=2)
# dataiter = iter(mnist_loader)

In [7]:
datapath = "/content/drive/MyDrive/A2_data/" #already shuffled n flattened data
traindata = np.load(datapath + "grey_traindata.npy", mmap_mode='r')
trainlabels = np.load(datapath + "grey_trainlabel.npy", mmap_mode='r')
testdata = np.load(datapath + "grey_testdata.npy", mmap_mode='r')
testabels = np.load(datapath + "grey_testlabel.npy", mmap_mode='r')

In [8]:
indxs = np.random.choice(len(traindata),int(len(traindata)/4))
traindata = traindata[indxs]

In [9]:
indxs = np.random.choice(len(testdata),int(len(testdata)/4))
testdata = testdata[indxs]

if want MNIST, begin

In [36]:
data = np.genfromtxt("/content/sample_data/mnist_train_small.csv", skip_header=True,delimiter=",")

In [37]:
traindata = []
trainlabels = []

i = 0
for val in data:
  if(i<10000):
    traindata.append(val[1:])
    trainlabels.append(val[0])
  i+=1

In [40]:
data = np.genfromtxt("/content/sample_data/mnist_test.csv", skip_header=True,delimiter=",")

In [17]:
print(len(data))

9999


In [41]:
testdata = []
testlabels = []
i = 0
for val in data:
  if(i<5000):
    testdata.append(val[1:])
    testlabels.append(val[0])
  i+=1

mnist, end

In [42]:
mu = np.mean(traindata,axis=0)
std = np.std(traindata,axis=0)

In [43]:
traindata1 = (traindata - mu)/(std+10**-5) 
testdata1 = (testdata - mu)/(std+10**-5) 

In [45]:
ncls = 10
trainlabels1 = []
for label in trainlabels:
  v = np.zeros(shape=(ncls,))
  #print(label)
  v[int(label)] = 1
  trainlabels1.append(v)
  #print(v)

#  break
# trainlabels = np.asarray(trainlabels1)
# del trainlabels1
# print(trainlabels.shape)

In [46]:
testlabels1 = []
for label in testlabels:
  v = np.zeros(shape=(ncls,))
  v[int(label)] = 1
  testlabels1.append(v)

# testabels = np.asarray(testlabels1)
# del testlabels1
# print(testabels.shape)

In [47]:
univ_seed = 42
np.random.seed(univ_seed)
#reproducibility

In [62]:
class layer:
  #layer object - each is a FC layer 

  def __init__(self,num_in,num_out,weights=None,b=None,activation=None):
    
    self.shapeval = (num_in,num_out)
    self.activation = activation
    self.wts = np.random.randn(num_in,num_out) if weights is None else weights
    self.bias = np.random.randn(num_out) if b is None else b
    self.err = 0
    self.diffval = 0
    self.penalty_norm = None
    self.activation_memo = None

  def apply_activation(self,val,some_other=None):
    if self.activation is None:
      return val

    #add activations inside class for speed
    if self.activation == "sigmoid":
      return 1/(1.0+np.exp(-val))
    
    if self.activation == "softmax":
      e = np.exp(val)
      return e / np.sum(e)

    if self.activation == "tanh":
      return np.tanh(val)

    if self.activation == "relu":
      return np.maximum(val, 0)

    else: #undefined activation
      if some_other is not None:
        #of the form activation(input)
        return some_other(val)

    return val 

  def activation_derivative(self,val,some_other=None):
    if self.activation == None:
      return val
    
    if self.activation == "sigmoid":
      tmp = 1/(1.0+np.exp(-val))
      return tmp*(1-tmp)
    
    if self.activation == "tanh":
      v = np.tanh(val)
      return 1 - val**2
    
    if self.activation == "softmax":
      e = np.exp(val)
      softmax = e / np.sum(e)
      s = softmax.reshape(1,-1)      
      return np.diagflat(s) - np.dot(s, s.T)
    
    if self.activation == "relu":
      return 1. * (val > 0)

    else: #undefined activation
      if some_other is not None:
        #of the form activation_derivative(input)
        return some_other(val)
      
    return val

  def after_apply_activ(self,val):
    
    net_out = np.dot(val,self.wts) + self.bias
    self.activation_memo = self.apply_activation(net_out)
    return self.activation_memo
  

In [49]:
class FNN:
  #FNN object - describes a FNN/MLP
  def __init__(self):
 
    self.network = list()
    #---better to make a diff layer class-----
    # self.network.append(np.random.random(size=(num_feat,num_hide[0])))
    # self.network_bias.append(np.ones(size=(num_hide[0],1)))

    # for i in range(0,len(num_hide)-1):
    #   self.network.append(np.random.random(size=(num_hide[i],num_hide[i+1])))
    #   self.network_bias.append(np.ones(size=(num_hide[i+1],1)))
    
    # self.network.append(np.random.random(size=(num_hide[-1],num_out)))
    # self.network_bias.append(np.ones(size=(num_out,1)))
    # 
    # self.activations = []

  def build_fnn(self,num_feat,num_hide,num_out,activns):
#     assert len(num_hide)+1 == len(activns)
     self.network = list()

     lay_in = layer(num_feat,num_hide[0],activation=activns[0])
     self.network.append(lay_in)
     
     for j in range(0,len(num_hide)-1):
       self.network.append(layer(num_hide[j],num_hide[j+1],activation=activns[j+1]))
       #pass

     self.network.append(layer(num_hide[-1],num_out,activation=activns[-1]))
     return 0

  def add_layer(self,num_in,num_out,activn,is_out=False):
    if(is_out): #if it is supposed to be new output layer
      v = self.network[-1].shapeval[1]
      assert v==num_in
      self.network.append(layer(num_in,num_out,activation=activn))
    
    else: #num in and num out are just useless
      vin = self.network[-2].shapeval[1]
      vout = self.network[-1].shapeval[0]
      self.network.append(layer(vin,vout,activation=activn))
      v11 = self.network[-2]
      self.network[-2] = self.network[-1]
      self.network[-1] = v11

  def loss(self,x,truev,loss_fn="mse"):

    if(loss_fn=="mse"):
        return 0.5*np.mean(np.square(x-truev))
    return 0
  
  def loss_der(self,x,truev,loss_fn="mse"):
    if(loss_fn=="mse"):
      return np.mean((x-truev))
    return 0
    #pass

  def fwd_pass(self,inp):
    for lay in self.network:
      inp = lay.after_apply_activ(inp)
    return inp

  def dropout(self,X, drop_probability):
      keep_probability = 1 - drop_probability
      mask = np.random.uniform(0, 1.0, X.shape) < keep_probability
      scale = 0.0
      if keep_probability > 0.0:
          scale = (1/keep_probability)

      return mask * X * scale

  def backprop(self,inp,truev,eta,regularizer,loss_fn):
    outp = self.fwd_pass(inp)

    #backward pass: find diffvals
    for i in range(len(self.network)):
      lay_inx = len(self.network)-i-1
      layr = self.network[lay_inx]

      if(i==0): #implies last/op layer
        layr.err = self.loss_der(outp,truev,loss_fn)
        layr.diffval = (layr.err)*layr.activation_derivative(layr.activation_memo)

      else:
        layr_back = self.network[lay_inx+1]
        layr.err = np.dot(layr_back.wts,layr_back.diffval)
        layr.diffval = layr.err*layr.activation_derivative(layr.activation_memo)        
      
      self.network[lay_inx] = layr
      
      if regularizer is not None:
        if(regularizer=="L1"):
   #       print(layr.wts.shape)
          lamd_v = 0.5
          tempv = np.ones(shape=layr.wts.shape)
          for val in range(layr.wts.shape[0]):
            for v1 in range(len(layr.wts[val])):
              if(layr.wts[val][v1]<0):
                tempv[val][v1] = -1
              if(layr.wts[val][v1]==0):
                tempv[val][v1] = 0
              
          layr.penalty_norm = lamd_v*np.sum(tempv,axis=0)
          del(tempv)

        if(regularizer=="L2"):
          lamd_v = 0.5
        #  print(layr.wts.shape)
          layr.penalty_norm = lamd_v*np.sum(layr.wts,axis=0)
       #   print(layr.penalty_norm.shape)

        if(regularizer=="dropout"):
          for lr in self.network:
            lr.wts = self.dropout(lr.wts,0.2)
          layr.penalty_norm = None
      
      if layr.penalty_norm is not None:
      #    print(layr.diffval.shape)
      #    print(layr.penalty_norm.shape)
          layr.diffval += layr.penalty_norm
      
      self.network[lay_inx] = layr

    #fwd pass: weight update
    for i in range(len(self.network)):
      if(i!=0):
        mtrx = np.atleast_2d(self.network[i-1].activation_memo)
      else:
        mtrx = np.atleast_2d(inp)

      self.network[i].wts -= self.network[i].diffval*mtrx.T*eta

  def predict(self,inpt,truev):
    outp = self.fwd_pass(inpt)
    if np.argmax(outp) == np.argmax(truev):
      return True
    #else:
    return False

  def find_acc(self,outp,truev):
    all_v = 0
    truevs = 0

    for inx1 in range(len(outp)):
      outpv = outp[inx1]
      #print(outpv)
      #print(truev)

      truevv = truev[inx1]
      
      bn = self.predict(outpv,truevv)
      if(bn):
        truevs+=1
      all_v +=1
    print(truevs)
    print(all_v)
    return round(float(truevs/all_v),3)    

  def train_fnn(self,testin,testtrue,inpt,truev,eta,maxiter,batch_size=32,regularizer=None,loss_fn="mse"):
    errs = []
    tracc = []
    testacc = []

    if(batch_size!=0):
      num_bins = int(len(inpt)/batch_size)
    else:
      num_bins = 1
    eps = 10**-5
    
    for iter in range(maxiter):
      err1tmp = []  
      minb_inx = np.random.randint(0,num_bins)

      if(batch_size!=0):
        inxsl = minb_inx*batch_size
        inpt1 = inpt[inxsl:] if minb_inx==num_bins-1 else inpt[inxsl:inxsl+batch_size]
        truev1 = truev[inxsl:] if minb_inx==num_bins-1 else truev[inxsl:inxsl+batch_size]
      else:
        inpt1 = inpt
        truev1 = truev

      if(regularizer=="batch"):
        mu1 = np.mean(inpt1)
        sd1 = np.std(inpt1)
        inpt1 = (inpt1-mu)/(sd1 + eps)
      
      for j in range(len(inpt1)):
        self.backprop(inpt1[j],truev1[j],eta,regularizer,loss_fn)
        err1tmp.append(self.loss(self.fwd_pass(inpt1[j]),truev1[j],loss_fn))

        
      errs.append(np.mean(np.asarray(err1tmp)))
      print(err1tmp)
      print("Epoch: %s, Loss: %f" % (iter,errs[-1]))  
      
      if(iter%5==0):
        tracc.append(self.find_acc(inpt,truev))
        testacc.append(self.find_acc(testin,testtrue))

    # checkpoints// commented
    # with open(datapath+"vals.pkl",'wb') as pf:
    #     pickle.dump(nn.network,pf,pickle.HIGHEST_PROTOCOL) wt saving so code can be resumed incase of BT

    print("accuracy: %f" %(self.find_acc(inpt,truev)))
    
    return errs,tracc,testacc
  


In [30]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=10, svd_solver='full')
# pca.fit(traindata1.T)

In [ ]:
nn = FNN()
# nn.build_fnn(len(traindata1[0]),[800,500,250],ncls,["tanh","sigmoid","sigmoid","tanh","sigmoid","sigmoid"]) --> MNIST
# nn.build_fnn(pca.components_,[10,10],ncls,["tanh","sigmoid","sigmoid","tanh","sigmoid","sigmoid"]) --> with pca if want to experiment
nn.build_fnn(len(traindata1[0]),[800,500,100,50],ncls,["tanh","sigmoid","sigmoid","sigmoid","sigmoid","sigmoid"]) #build and init weights

In [ ]:
errs,tr,ts = nn.train_fnn(testdata1,testlabels1,traindata1,trainlabels1,10**-3,100,batch_size=0,regularizer="L2") #0 batch size means no minibatches

In [ ]:
# print(errs)
# print(tr)
# print(ts)

In [ ]:
# ----code for visualization - keep modifyig while generating plots
# import matplotlib.pyplot as plt
# plt.plot(range(epoch),errs,label="train")
# # plt.plot(range(0,100,10),ts,label="test")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.title("FNN, MSE loss, lr 10^-6")